In [10]:
import numpy as np
import pandas as pd
from ipywidgets import interact_manual, interact
from ipywidgets import Button, HBox, IntText, HTML, VBox
from IPython.display import display

# Chargement des données 

On charge les propositions présidentielles.

In [12]:
def load_data():
    "Loads presidential candidates data."
    mlp = pd.read_csv('projets/marine_le_pen.csv')['texte de la proposition'].str.replace('^.\s', '')
    mlp.name = 'proposition'
    yj = pd.read_csv('projets/yannick_jadot.csv')['proposition'].str.replace(' →', '')
    bh = pd.read_csv('projets/benoit_hamon.csv')['contenu']
    bh.name = 'proposition'
    jlm = pd.read_csv('projets/jean_luc_melenchon.csv')['mesures'].dropna().str.replace("Nous proposons de réaliser les mesures suivantes\xa0:", '').replace('\xa0', '  ')
    jlm.name = 'proposition'
    em = pd.read_csv('projets/emmanuel_macron.csv')['proposition']
    candidates = [jlm, yj, bh, em, mlp]
    candidate_labels = ['JLM', 'YJ', 'BH', 'EM', 'MLP']
    return candidates, candidate_labels

In [13]:
candidates, candidate_labels = load_data()

# Quizz candidats 

In [8]:
class GUI:
    def __init__(self):
        self.labels = candidate_labels

        buttons = [Button(description=label) for label in self.labels]
        for b in buttons:
            b.on_click(self.on_button_clicked)
        hbox1 = HBox()
        hbox1.children = buttons
        self.buttons = buttons
        
        self.scorebox = IntText(description='score', value=0)
        new_question = Button(description='nouvelle question !')
        new_question.on_click(self.create_new_question)
        hbox2 = HBox()
        hbox2.children = [self.scorebox, new_question]

        self.html = HTML()
        
        vbox = VBox()
        vbox.children = [hbox1, hbox2, self.html]
        self.box = vbox

        self.create_new_question()
        
    def create_new_question(self, b=None):
        choice = np.random.randint(0, len(candidates))
        self.choice = choice
        candidate_series = candidates[choice]
        n = candidate_series.size
        self.html.value = '<p style="font-size: 150%;font-weight: 300;line-height: 1.39;margin: 0 0 12.5px;">{}</p>'.format(candidate_series[np.random.randint(n)])
        for b in self.buttons:
            b.style.button_color = None
        
    def on_button_clicked(self, b):
        if b.description == self.labels[self.choice]:
            self.scorebox.value += 1
            b.style.button_color = 'lightgreen'
        else:
            self.scorebox.value -= 1
            b.style.button_color = 'red'


In [11]:
gui = GUI()
display(gui.box)

In [ ]:
def make_df(candidate, candidate_label):
    "Returns a dataframe with candidate name."
    ind_s = pd.Series(index=candidate.index, data=[candidate_label], name='candidat')
    return pd.DataFrame(data=[ind_s, candidate]).transpose()

In [ ]:
all_propositions = pd.concat([make_df(c, label) for c, label in zip(candidates, candidate_labels)])

In [ ]:
ind = all_propositions['proposition'].str.lower().str.contains('vert')

In [ ]:
all_propositions[ind]

In [ ]:
mask = all_propositions[ind]

In [ ]:
mask.groupby(by='candidat').sum().iloc[0].values

In [ ]:
from ipywidgets import interact, IntSlider, SelectMultiple

In [ ]:
mask.size

In [ ]:
slider = IntSlider(min=0, max=mask.size - 1)
slider

In [ ]:
selector = SelectMultiple(
        options=mask['proposition'].values.tolist(),
        value=[],
        description='',
        disabled=False,
        )
selector.layout.width='500px'
selector.layout.height = "300px"

In [ ]:
@interact
def search_for_props(search='croissance'):
    ind = all_propositions['proposition'].str.lower().str.contains(search)
    mask = all_propositions[ind]
    selector.options = mask['proposition'].values.tolist()

display(selector)